In [1]:
import json
import pandas as pd
import numpy as np



In [2]:

!git clone https://huggingface.co/datasets/newbienewbie/handwriting_strokes



Cloning into 'handwriting_strokes'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 85 (delta 21), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (85/85), 21.97 KiB | 271.00 KiB/s, done.


In [3]:
file_path = "/content/handwriting_strokes/data/test-00000-of-00001.parquet"
testData = pd.read_parquet(file_path)

file_path = "/content/handwriting_strokes/data/train-00000-of-00001.parquet"
trainData = pd.read_parquet(file_path)

file_path = "/content/handwriting_strokes/data/validation-00000-of-00001.parquet"
validationData = pd.read_parquet(file_path)


# Inspect the strokes column
# print(testData)

In [4]:

from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Extracting x, y, time, and annotations
xCoordinates = [np.array(item, dtype=np.float32) for item in trainData['xCoordinates'].tolist()]
yCoordinates = [np.array(item, dtype=np.float32) for item in trainData['yCoordinates'].tolist()]
# timeStamps = [np.array(item, dtype=np.float32) for item in trainData['timeStamps'].tolist()]
annotations = [str(item) for item in trainData['sampleTag']]



# Normalize coordinates and timestamps
scaler = MinMaxScaler()

xCoordinates = [scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in xCoordinates]
yCoordinates = [scaler.fit_transform(y.reshape(-1, 1)).flatten() for y in yCoordinates]
# timeStamps = [scaler.fit_transform(t.reshape(-1, 1)).flatten() for t in timeStamps]

# Combine and pad sequences
combinedData = [np.column_stack((x, y)) for x, y in zip(xCoordinates, yCoordinates)]
# print(combinedData)
print(xCoordinates[0])

[0.98611104 0.5416666  0.4722222  0.4027778  0.3472222  0.30555558
 0.26388884 0.20833337 0.16666663 0.125      0.08333337 0.05555558
 0.01388884 0.         0.         0.         0.         0.
 0.         0.         0.         0.02777779 0.11111116 0.15277779
 0.20833337 0.25       0.2777778  0.31944442 0.3472222  0.38888884
 0.43055558 0.5277778  0.6527778  0.6944444  0.73611116 0.7777778
 0.8055556  0.86111116 0.88888884 0.9166666  0.9444444  0.9583334
 0.98611104 1.0000001  1.0000001  1.0000001  1.0000001  1.0000001
 1.0000001  1.0000001  1.0000001  0.9722222  0.9305556  0.9027778
 0.875      0.8194444  0.8055556  0.7777778  0.75       0.73611116
 0.73611116 0.7777778 ]


In [5]:
def select_model_20(item):
    if 10 < item.shape[0] < 30:
        return True
    else:
        return False

def select_model_40(item):
    if 30 < item.shape[0] < 50:
        return True
    else:
        return False

def select_model_60(item):
    if 50 < item.shape[0] < 70:
        return True
    else:
        return False

def select_model_80(item):
    if 70 < item.shape[0] < 90:
        return True
    else:
        return False


In [6]:
filtered_20 = []
annotations_20 = []
for index, data in enumerate(combinedData):
    if select_model_20(data):
        filtered_20.append(data)
        annotations_20.append(annotations[index])
# print(len(filtered_20))


filtered_40 = []
annotations_40 = []

for index, data in enumerate(combinedData):
    if select_model_40(data):
        filtered_40.append(data)
        annotations_40.append(annotations[index])

# To check the results
print(annotations_40)

filtered_60 = []
annotations_60 = []

for index, data in enumerate(combinedData):
    if select_model_60(data):
        filtered_60.append(data)
        annotations_60.append(annotations[index])

# To check the results
print(annotations_60)

['2', '5', '12', '2', '4', '5', '8', '10', '12', '16', '5', '8', '12', '15', '16', '5', '8', '10', '15', '16', '10', '12', '15', '16', '4', '5', '8', '10', '12', '14', '15', '16', '2', '5', '8', '10', '12', '14', '16', '2', '4', '8', '10', '2', '5', '10', '16', '0', '2', '4', '7', '8', '10', '12', '0', '2', '4', '5', '8', '10', '11', '14', '0', '2', '4', '8', '10', '11', '0', '2', '3', '4', '5', '8', '10', '11', '14', '0', '2', '4', '7', '8', '9', '10', '11', '0', '2', '4', '8', '10', '11', '12', '2', '3', '5', '8', '10', '12', '0', '2', '4', '10', '0', '2', '3', '4', '7', '8', '10', '14', '2', '4', '8', '10', '12', '18', '0', '2', '4', '8', '10', '11', '12', '2', '4', '5', '8', '10', '11', '17', '18', '0', '2', '3', '4', '10', '11', '12', '17', '0', '2', '4', '9', '11', '12', '14', '17', '18', '0', '2', '3', '4', '8', '9', '10', '11', '12', '18', '0', '2', '3', '4', '10', '11', '12', '14', '2', '3', '4', '9', '14', '2', '3', '4', '8', '10', '12', '0', '2', '3', '4', '7', '10', '12', '

In [7]:

max_len = 30
padded_data_20 = pad_sequences(filtered_20, maxlen=max_len, padding='post', dtype='float32')


max_len = 50
padded_data_40 = pad_sequences(filtered_40, maxlen=max_len, padding='post', dtype='float32')


max_len = 70
padded_data_60 = pad_sequences(filtered_60, maxlen=max_len, padding='post', dtype='float32')



# One-hot encode annotations
label_binarizer = LabelBinarizer()

all_annotations = annotations_20 + annotations_40 + annotations_60  # Combine all annotations
label_binarizer = LabelBinarizer()
label_binarizer.fit(all_annotations)
labels_20 = label_binarizer.transform(annotations_20)
labels_40 = label_binarizer.transform(annotations_40)
labels_60 = label_binarizer.transform(annotations_60)

# print(labels_60)

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(padded_data_20.shape[1], padded_data_20.shape[2])))
model.add(LSTM(50))
model.add(Dense(labels_20.shape[1], activation='softmax'))  # Number of classes

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_data_20, labels_20, epochs=10, batch_size=1)

print(labels_20)

# To evaluate the model, prepare the test data similarly and use model.evaluate(test_data, test_labels)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2627 - loss: 2.6727
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4604 - loss: 1.7535
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4785 - loss: 1.4436
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5900 - loss: 1.0670
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6408 - loss: 0.9355
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5641 - loss: 0.9014
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8252 - loss: 0.5944
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8155 - loss: 0.5474
Epoch 9/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8187 - loss: 0.5742
Epoch 10/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7425 - loss: 0.6166
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.optimizers import Adam

# Define the model
model_40 = Sequential()
model_40.add(Masking(mask_value=0.0, input_shape=(padded_data_40.shape[1], padded_data_40.shape[2])))
model_40.add(LSTM(50))
model_40.add(Dense(labels_40.shape[1], activation='softmax'))  # Number of classes

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model_40.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_40.fit(padded_data_40, labels_40, epochs=10, batch_size=1)

# print(labels_20)

# To evaluate th

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0714 - loss: 2.8595
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.2945 - loss: 2.0657
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.4697 - loss: 1.6736
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5667 - loss: 1.4627
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6608 - loss: 1.0827
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6496 - loss: 1.1360
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6238 - loss: 1.0561
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6750 - loss: 0.8897
Epoch 9/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.6677 - loss: 0.9361
Epoch 10/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8114 - loss: 0.6503


In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.optimizers import Adam

# Define the model
model_60 = Sequential()
model_60.add(Masking(mask_value=0.0, input_shape=(padded_data_60.shape[1], padded_data_60.shape[2])))
model_60.add(LSTM(50))
model_60.add(Dense(labels_60.shape[1], activation='softmax'))  # Number of classes

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model_60.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_60.fit(padded_data_60, labels_60, epochs=10, batch_size=1)

# print(labels_20)

# To evaluate th

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.1328 - loss: 2.8091
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.4734 - loss: 1.8149
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.5701 - loss: 1.2833
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6459 - loss: 1.1295
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.7519 - loss: 0.8670
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.7021 - loss: 0.8510
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6164 - loss: 1.0664
Epoch 8/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.8116 - loss: 0.6761
Epoch 9/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7339 - loss: 0.7106
Epoch 10/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8900 - loss: 0.3697


In [22]:

# Testing Extracting x, y, time, and annotations

# Extracting x, y, time, and annotations
xCoordinates_t = [np.array(item, dtype=np.float32) for item in testData['xCoordinates'].tolist()]
yCoordinates_t = [np.array(item, dtype=np.float32) for item in testData['yCoordinates'].tolist()]
# timeStamps_t = [np.array(item, dtype=np.float32) for item in testData['timeStamps'].tolist()]
annotations_t = [str(item) for item in testData['sampleTag']]



# Normalize coordinates and timestamps
scaler = MinMaxScaler()

xCoordinates_t = [scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in xCoordinates_t]
yCoordinates_t = [scaler.fit_transform(y.reshape(-1, 1)).flatten() for y in yCoordinates_t]
# timeStamps_t = [scaler.fit_transform(t.reshape(-1, 1)).flatten() for t in timeStamps_t]

# Combine and pad sequences
combinedData_t = [np.column_stack((x, y)) for x, y in zip(xCoordinates_t, yCoordinates_t)]
# print(combinedData)

In [23]:
filtered_20_t = []
annotations_20_t = []
for index, data in enumerate(combinedData_t):
    if select_model_20(data):
        filtered_20_t.append(data)
        annotations_20_t.append(annotations_t[index])
print(filtered_20_t)


filtered_40_t = []
annotations_40_t = []

for index, data in enumerate(combinedData_t):
    if select_model_40(data):
        filtered_40_t.append(data)
        annotations_40_t.append(annotations_t[index])

# To check the results
print(annotations_40_t)

filtered_60_t = []
annotations_60_t = []

for index, data in enumerate(combinedData_t):
    if select_model_60(data):
        filtered_60_t.append(data)
        annotations_60_t.append(annotations_t[index])

# To check the results
print(annotations_60_t)

[array([[0.        , 0.        ],
       [0.        , 0.37681162],
       [0.        , 0.5072464 ],
       [0.        , 0.5652174 ],
       [0.        , 0.63043475],
       [0.        , 0.68840575],
       [0.        , 0.7391305 ],
       [0.        , 0.826087  ],
       [0.        , 0.86956525],
       [0.20000076, 0.89855075],
       [0.20000076, 0.942029  ],
       [0.20000076, 0.9637681 ],
       [0.20000076, 0.9782609 ],
       [0.20000076, 0.9927536 ],
       [0.20000076, 1.        ],
       [0.39999962, 1.        ],
       [1.        , 0.98550725]], dtype=float32), array([[0.9399998 , 0.        ],
       [0.03999996, 0.40441176],
       [0.01999998, 0.4485294 ],
       [0.        , 0.49264708],
       [0.        , 0.53676474],
       [0.        , 0.57352936],
       [0.        , 0.6102941 ],
       [0.        , 0.64705884],
       [0.01999998, 0.6838236 ],
       [0.05999994, 0.7205882 ],
       [0.13999987, 0.75735295],
       [0.22000003, 0.7941177 ],
       [0.31999993, 0.823

In [ ]:
# max_len = 30
# padded_data_t = pad_sequences(filtered_20_t, maxlen=max_len, padding='post', dtype='float32')

# # One-hot encode annotations
# label_binarizer = LabelBinarizer()
# labels_t = label_binarizer.fit_transform(annotations_20_t)
# print(labels_20.shape)
# print(labels_t.shape)

# model.evaluate(padded_data_t, labels_t)

(41, 16)
(13, 13)


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 13), output.shape=(None, 16)

In [15]:
def predict_sample(input_data):
    # Preprocess the input
    padded_input = pad_sequences([input_data], maxlen=max_len, padding='post', dtype='float32')

    # Predict
    prediction = model.predict(padded_input)
    predicted_class = np.argmax(prediction, axis=1)

    # Map to label
    label = label_binarizer.inverse_transform(prediction)
    return label

In [24]:
for i, test_data in enumerate(filtered_20_t):
    max_len = 30
    # Pad the single sample
    test_data_padded = pad_sequences([test_data], maxlen=max_len, padding='post', dtype='float32')

    # Predict
    prediction = model.predict(test_data_padded)


    # Interpret
    predicted_label = label_binarizer.inverse_transform(prediction)
    print(f"Sample {i}: Predicted label: {predicted_label}")
    print(f"Sample {i}: Predicted Value: {prediction[0][i]}")
    print(f"Sample {i}: Actual label: {annotations_20_t[i]}")
print(annotations_20_t)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Sample 0: Predicted label: ['1']
Sample 0: Predicted Value: 0.0001682336733210832
Sample 0: Actual label: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Sample 1: Predicted label: ['15']
Sample 1: Predicted Value: 0.007550232112407684
Sample 1: Actual label: 15
['1', '15']


In [26]:
import matplotlib.pyplot as plt
for i, test_data in enumerate(filtered_40_t):

    max_len = 50
    # Pad the single sample
    test_data_padded = pad_sequences([test_data], maxlen=max_len, padding='post', dtype='float32')

    # Predict
    prediction = model_40.predict(test_data_padded)


    # Interpret
    predicted_label = label_binarizer.inverse_transform(prediction)
    print(f"Sample {i}: Predicted label: {predicted_label}")
    print(f"Sample {i}: Predicted Value: {prediction[0][i]}")
    print(f"Sample {i}: Actual label: {annotations_40_t[i]}")

    # Define grid size (rows, columns)
    grid_rows = 100  # Set the number of rows in your grid
    grid_cols = 100  # Set the number of columns in your grid

    # Map the normalized coordinates to the grid
    mapped_coordinates = np.zeros_like(test_data)
    mapped_coordinates[:, 0] = test_data[:, 0] * (grid_rows - 1)  # Map x to rows
    mapped_coordinates[:, 1] = test_data[:, 1] * (grid_cols - 1)  # Map y to columns

    # Plotting the graph
    # plt.figure(figsize=(2, 1.5))
    # plt.scatter(mapped_coordinates[:, 0], mapped_coordinates[:, 1], c='blue', label='Mapped Coordinates')

    # # Adding labels and title
    # plt.xlabel('X Coordinates (Rows)')
    # plt.ylabel('Y Coordinates (Columns)')
    # plt.title('Mapped Coordinates on Grid')

    # Show the grid
    # plt.grid(True)

    # Show the plot
    # plt.legend()
    # plt.show()
    # print(mapped_coordinates)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sample 0: Predicted label: ['2']
Sample 0: Predicted Value: 0.0006056834827177227
Sample 0: Actual label: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sample 1: Predicted label: ['4']
Sample 1: Predicted Value: 3.1428313377546147e-05
Sample 1: Actual label: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sample 2: Predicted label: ['8']
Sample 2: Predicted Value: 0.0012181242927908897
Sample 2: Actual label: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Sample 3: Predicted label: ['0']
Sample 3: Predicted Value: 0.35973936319351196
Sample 3: Actual label: 10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sample 4: Predicted label: ['12']
Sample 4: Predicted Value: 0.8579062223434448
Sample 4: Actual label: 12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sample 5: Predicted label: ['2']
Sample 5: Predicted Value: 6.889382348163053e-05
Sample 5: Actual label: 14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Sample 6: Predicted label: ['5']
Sample 6: Predicted Value: 0.009052947163581848
Sampl

In [29]:
for i, test_data in enumerate(filtered_60_t):
    max_len = 70
    # Pad the single sample
    test_data_padded = pad_sequences([test_data], maxlen=max_len, padding='post', dtype='float32')

    # Predict
    prediction = model_60.predict(test_data_padded)


    # Interpret
    predicted_label = label_binarizer.inverse_transform(prediction)
    print(f"Sample {i}: Predicted label: {predicted_label}")
    print(f"Sample {i}: Predicted Value: {prediction[0][i]}")
    print(f"Sample {i}: Actual label: {annotations_60_t[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Sample 0: Predicted label: ['11']
Sample 0: Predicted Value: 0.34572261571884155
Sample 0: Actual label: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Sample 1: Predicted label: ['3']
Sample 1: Predicted Value: 8.495241672790144e-06
Sample 1: Actual label: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Sample 2: Predicted label: ['7']
Sample 2: Predicted Value: 0.0001620188995730132
Sample 2: Actual label: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Sample 3: Predicted label: ['9']
Sample 3: Predicted Value: 0.19875609874725342
Sample 3: Actual label: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Sample 4: Predicted label: ['11']
Sample 4: Predicted Value: 5.036158472648822e-05
Sample 4: Actual label: 11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Sample 5: Predicted label: ['13']
Sample 5: Predicted Value: 0.9535701274871826
Sample 5: Actual label: 13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Sample 6: Predicted label: ['18']
Sample 6: Predicted Value: 0.0006207710248418152
Samp

In [ ]:
print(filtered_20_t)

[[0.78947353 0.         0.        ]
 [0.         0.277027   0.5295048 ]
 [0.05263138 0.3648649  0.55895317]
 [0.05263138 0.4391892  0.58858615]
 [0.05263138 0.5135135  0.61751527]]
